一旦仮で作ったところまで

In [1]:
import pandas as pd

# データAとデータBのCSVファイルのパス
data_a_path = '/Users/whale217/dev/ProjectTeam5/workspace/team1/data/original_data/race_table/combined.csv'  # データAのCSVファイルパス
data_b_path = '/Users/whale217/dev/ProjectTeam5/workspace/team1/data/merged_data/merged_sorted.csv'  # データBのCSVファイルパス

# CSVファイルからデータを読み込む
data_a = pd.read_csv(data_a_path,low_memory=False)
data_b = pd.read_csv(data_b_path)
data_a=data_a.dropna()
data_b=data_b.dropna()
data_a=data_a.drop(columns=['first_horses', 'second_horses', 'third_horses', 'Time', 'Jockey', 'Trainer'])
# Code列でデータを結合 (data_bにdata_aの情報を追加)
merged_data = pd.merge(data_b, data_a, on='Code', how='left')

#結果の確認
merged_data.to_csv('../notebook/data-shin/merged_data_final.csv', index=False)
# 各列のユニークな値とその出現回数を取得
for i in merged_data.columns:
    value_counts = merged_data[i].value_counts()  # 各値とそのカウント
    print(f"{i}: {value_counts}")




KeyboardInterrupt: 

なんかパスが奇妙なエラーを吐くので一旦絶対パス使ってます。

In [1]:
import matplotlib.pyplot as plt
print("matplotlib is successfully installed!")


matplotlib is successfully installed!


In [3]:
import pandas as pd
import numpy as np

# データの読み込み（エンコーディングを指定して文字化け防止）
file_path = '/Users/whale217/dev/ProjectTeam5/workspace/team1/notebook/data-shin/merged_data_final.csv'
df = pd.read_csv(file_path, encoding='utf-8', low_memory=False)

# 1. Sex/Ageを分割して新しい列に追加
df[['Sex', 'Age']] = df['Sex/Age'].str.extract(r'([A-Za-z]+)(\d+)')
df['Age'] = pd.to_numeric(df['Age'], errors='coerce')  # 数値型に変換、変換できないものはNaNに
df.drop(columns=['Sex/Age'], inplace=True)  # 元のSex/Age列を削除

# 2. カテゴリカルデータをOne-Hot Encoding
categorical_columns = ['Sex', 'Jockey', 'Track', 'Weather', 'Condition', 'Trainer', 'Banushi', 'Horse Name']
df = pd.get_dummies(df, columns=categorical_columns, dtype=np.uint8)  # メモリ効率の良い型を指定

# 3. 欠損値の処理（欠損値を削除、文字列データに対してNaNを取り扱う）
df.dropna(subset=['Sex', 'Age'], inplace=True)  # SexやAgeに欠損があれば削除
df.dropna(inplace=True)  # その他の欠損値がある行を削除

# 4. インデックスの異常データを削除（インデックスが重複している行を削除）
df = df.loc[~df.index.duplicated(), :]  # 重複するインデックスを削除

# 5. メモリの最適化（データ型の変更）
# 数値型のカラムに対して適切な型に変換
for col in df.select_dtypes(include=['float64']).columns:
    df[col] = pd.to_numeric(df[col], downcast='float')
for col in df.select_dtypes(include=['int64']).columns:
    df[col] = pd.to_numeric(df[col], downcast='integer')

# 6. 不要な列の削除（CodeやRankなど予測に不要なら削除）
columns_to_drop = ['Code', 'Rank', 'Frame Rank']  # 必要に応じて列を指定
df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# 7. 前処理後のデータを新しいCSVファイルに保存
output_path = 'processed_horse_data_optimized.csv'  # 保存するファイル名を指定
df.to_csv(output_path, index=False)

print(f"前処理が完了しました。処理済みデータは {output_path} に保存されました。")


: 

features = [
    "Rank","Kinryou","Nobori","Ninki",
    "Race Number","Distance","Weight","Weight Change",
    "Sex","Age","Ground_ダ","Ground_芝","Ground_障","Condition_不",
    "Condition_稍","Condition_良","Condition_重","Weather_小雨","Weather_小雪",
    "Weather_晴","Weather_曇","Weather_雨","Weather_雪"
]
target = "Time_x"

In [10]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# データの読み込み
file_path = '/Users/whale217/dev/ProjectTeam5/workspace/team1/data/format_data.csv'
df = pd.read_csv(file_path)

print(df.columns)


# 必要な特徴量とターゲット変数の選択
X = df[[
    "Rank", "Kinryou", "Nobori", "Ninki",
    "Race Number", "Distance", "Weight", "Weight Change",
    "Sex", "Age", "Ground_ダ", "Ground_芝", "Ground_障", 
    "Condition_不", "Condition_稍", "Condition_良", "Condition_重", 
    "Weather_小雨", "Weather_小雪", "Weather_晴", "Weather_曇", "Weather_雨", "Weather_雪"
]]

y = df['Time_x']  # ここにターゲット変数の列名を指定

# 数値データが文字列として読み込まれている場合、カンマを取り除き、数値に変換
X = X.apply(lambda x: x.replace(',', '') if x.dtype == 'O' else x)  # カンマ削除
y = y.apply(lambda x: str(x).replace(',', '')).astype(float)  # カンマ削除後、float型に変換

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

# Lasso回帰モデルの作成と学習
lasso = Lasso(alpha=0.1)  # alpha は正則化の強さ
lasso.fit(X_train, y_train)

# 予測
y_pred = lasso.predict(X_test)

# 予測結果の表示
print(f'Mean Squared Error: {mean_squared_error(y_test, y_pred)}')


/var/folders/0y/s3g9vbzd3vvch0mp2zlmz2pr0000gn/T/ipykernel_1814/1498406558.py:8: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Index(['Code', 'Rank', 'Frame Rank', 'Horse Number', 'Horse Name', 'Kinryou',
       'Jockey_x', 'Time_x', 'Chakusa', 'Tsuuka', 'Nobori', 'Tansyou', 'Ninki',
       'Trainer_x', 'Banushi', 'Shoukin', 'Date', 'Track', 'Race Number',
       'Distance', 'Weight', 'Weight Change', 'Sex', 'Age', 'Ground_ダ',
       'Ground_芝', 'Ground_障', 'Condition_不', 'Condition_稍', 'Condition_良',
       'Condition_重', 'Weather_小雨', 'Weather_小雪', 'Weather_晴', 'Weather_曇',
       'Weather_雨', 'Weather_雪'],
      dtype='object')


ValueError: Input X contains NaN.
Lasso does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values